In [3]:
# import packages
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter, ParameterVector
from qiskit.visualization import plot_histogram
from qiskit_ionq import IonQProvider
import numpy as np

provider = IonQProvider("tQgNZln2nI3JSOg7hZhRXjSJHYfgrS2S")

provider.backends()
backend = provider.get_backend("ionq_simulator")

In [74]:
def random_walk(start, J_val, K_vals, steps):
    L = len(K_vals)
    #Define Parameters
    J = Parameter('J')
    K_i = []
    for ind in range(L):
        K_i.append(Parameter('K_'+str(ind)))
    #Define Unitary
    qc_U = QuantumCircuit(L)
    for i in range(L):
        qc_U.rz(K_i[i]/2, i)
    for i in range(L):
        if i % 2 == 0:
            qc_U.rxx(-J/2, i%L, (i+1)%L)
            qc_U.ryy(-J/2, i%L, (i+1)%L)
    for i in range(L):
        if i % 2 != 0:
            qc_U.rxx(-J, i%L, (i+1)%L)
            qc_U.ryy(-J, i%L, (i+1)%L)
    for i in range(L):
        if i % 2 == 0:
            qc_U.rxx(-J/2, i%L, (i+1)%L)
            qc_U.ryy(-J/2, i%L, (i+1)%L)
    for i in range(L):
        qc_U.rz(K_i[i]/2, i)
        #qc_U.p(K_i[i], i)
    #Compose Main Circuit with set number of Unitary steps:
    qc_main = QuantumCircuit(L)
    qc_main.x(start)
    for step in range(steps):
        qc_main = qc_main.compose(qc_U, qubits=range(L))
    #Add Measurement Circuit
    qc_meas = QuantumCircuit(L,L)
    qc_meas.measure_all(add_bits=False)
    qc_end = qc_main.compose(qc_meas, range(L))
    #Bind Parameters
    qc_end = qc_end.bind_parameters({J:J_val})
    for ind in range(L):
        qc_end = qc_end.bind_parameters({K_i[ind]:K_vals[ind]})
    #Transpile and Run
    trans_circ = transpile(qc_end, backend)
    job = backend.run(trans_circ)
    #Return the counts for the jobs
    return job.result().get_counts()

In [ ]:
def random_walk(start, J_val, J_val_2, K_vals, steps):
    L = len(K_vals)
    #Define Parameters
    J = Parameter('J')
    J2 = Parameter('J2')
    K_i = []
    for ind in range(L):
        K_i.append(Parameter('K_'+str(ind)))
    #Define Unitary
    qc_U = QuantumCircuit(L)
    for i in range(L):
        qc_U.rz(K_i[i]/2, i)
    #even - nearest neighbors
    for i in range(L):
        if i % 2 == 0:
            qc_U.rxx(-J/2, i%L, (i+1)%L)
            qc_U.ryy(-J/2, i%L, (i+1)%L)
    #odd - nearest neighbors
    for i in range(L):
        if i % 2 != 0:
            qc_U.rxx(-J/2, i%L, (i+1)%L)
            qc_U.ryy(-J/2, i%L, (i+1)%L)
            
    #evens - 2nd nearest neighbors
    for i in range(L):
        if i % 2 == 0:
            qc_U.rxx(-J2/2, i%L, (i+2)%L)
            qc_U.ryy(-J2/2, i%L, (i+2)%L)
    #odds - 2nd nearest neighbors
    for i in range(L):
        if i % 2 != 0:
            qc_U.rxx(-J2, i%L, (i+2)%L)
            qc_U.ryy(-J2, i%L, (i+2)%L)   
    #evens - 2nd nearest neighbors     
    for i in range(L):
        if i % 2 == 0:
            qc_U.rxx(-J2/2, i%L, (i+2)%L)
            qc_U.ryy(-J2/2, i%L, (i+2)%L)

    #odd - nearest neighbors
    for i in range(L):
        if i % 2 != 0:
            qc_U.rxx(-J/2, i%L, (i+1)%L)
            qc_U.ryy(-J/2, i%L, (i+1)%L)
    #even nearest neighbors
    for i in range(L):
        if i % 2 == 0:
            qc_U.rxx(-J/2, i%L, (i+1)%L)
            qc_U.ryy(-J/2, i%L, (i+1)%L)
    for i in range(L):
        qc_U.rz(K_i[i]/2, i)
        #qc_U.p(K_i[i], i)
    #Compose Main Circuit with set number of Unitary steps:
    qc_main = QuantumCircuit(L)
    qc_main.x(start)
    for step in range(steps):
        qc_main = qc_main.compose(qc_U, qubits=range(L))
    #Add Measurement Circuit
    qc_meas = QuantumCircuit(L,L)
    qc_meas.measure_all(add_bits=False)
    qc_end = qc_main.compose(qc_meas, range(L))
    #Bind Parameters
    qc_end = qc_end.bind_parameters({J:J_val})
    qc_end = qc_end.bind_parameters({J2:J_val_2})
    for ind in range(L):
        qc_end = qc_end.bind_parameters({K_i[ind]:K_vals[ind]})
    #Transpile and Run
    trans_circ = transpile(qc_end, backend)
    job = backend.run(trans_circ)
    #Return the counts for the jobs
    return job.result().get_counts()

In [99]:
start = 0
K_vals = 0*np.ones(12)
K_vals[0] = 0
steps = 5
J_val = np.pi/10


results = random_walk(start, J_val, K_vals, steps)
results

{'000000000001': 68,
 '000000000010': 19,
 '000000000100': 0,
 '000000001000': 1,
 '000000010000': 408,
 '000000100000': 23,
 '000001000000': 3,
 '000010000000': 32,
 '000100000000': 447,
 '001000000000': 2,
 '010000000000': 2,
 '100000000000': 19}

In [12]:
K_vals

array([ 0.,  0.,  0.,  0.,  0.,  0., 20.,  0.,  0.,  0.,  0.,  0.])